In [0]:
import dlt
import pyspark.sql.functions as F

In [0]:
# The schema location directory keeps track of your data schema over time
SCHEMA_LOCATION = "dbfs:/tmp/chp_02/taxi_data_chkpnt"

# The location where raw trip data is written
RAW_DATA_LOCATION = "dbfs:/tmp/chp_02/taxi_data/"

In [0]:
@dlt.table(
    name="raw_taxi_trip_data",
    comment="Raw taxi trip data generated by the data generator notebook",
)
def raw_taxi_trip_data():
    return (
        spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.schemaLocation", SCHEMA_LOCATION)
        .load(RAW_DATA_LOCATION)
    )

In [0]:
@dlt.table(
    name="raw_driver_data",
    comment="Dataset containing information about the taxi drivers",
)
def raw_driver_data():
    return spark.createDataFrame(
        data=[
            (10001, 7292, "Toyota", "Camry", 5),
            (10002, 7782, "Honda", "Civic", 4),
            (10001, 2736, "Toyota", "Corolla", 5),
            (10001, 5284, "Toyota", "Corolla", 5),
            (10001, 1922, "Toyota", "Corolla", 5),
            (10001, 1704, "Toyota", "Corolla", 5),
            (10001, 6560, "Ford", "Escape", 5),
            (10001, 2086, "Ford", "Crown Victoria", 5),
            (10001, 8266, "Ford", "Crown Victoria", 5),
            (10001, 2013, "Toyota", "Camry", 5),
            (10001, 2798, "Toyota", "Sienna", 7),
        ],
        schema="driver_id STRING, taxi_number INT, cab_make STRING, cab_model STRING, seating_capacity INT",
    )

In [0]:
# First, we'll create a target table to apply changes
dlt.create_streaming_table("taxi_trip_data_merged")

# Next, we'll define how the changes should be applied
# including the SCD type and which columns to ignore
dlt.apply_changes(
    target="taxi_trip_data_merged",
    source="raw_taxi_trip_data",
    keys=["trip_id"],
    sequence_by=F.col("sequence_num"),
    apply_as_deletes=F.expr("op_type = 'D'"),
    except_column_list=["op_type", "op_date", "sequence_num"],
    stored_as_scd_type=2,
)

In [0]:
# Lastly, we'll create a silver table with a few transformations
# Let's round the float columns to 2 decimals places as well as
# transform the trip distance column into columns for miles and kilometers
@dlt.table(name="taxi_trip_silver",
           comment="Taxi trip data with transformed columns")
def taxi_trip_silver():
    return (
        dlt.read("taxi_trip_data_merged")
        .withColumn("fare_amount_usd", F.round(F.col("trip_amount"), 2))
        .withColumn("taxes_amount_usd", F.round(F.col("trip_amount") * 0.05, 2))
        .withColumn("trip_distance_miles", F.round(F.col("trip_distance"), 2))
        .withColumn(
            "trip_distance_km", F.round(F.col("trip_distance") * 1.60934, 2)
        )  # 1 mile = 1.60934 km
    ).join(dlt.read("raw_driver_data"), on="taxi_number", how="left")